# Combineren en manipuleren van gegevens

De verzamelde data moet gecombineerd worden voordat deze gemodelleerd kan worden. De gegevens van SolarEdge, het weer en de stand van de zon komen in één tabel.

- Solaredge: Dates are always in the time zone of the site. De tijdstippen van SolarEdge worden (voor nu) met 1 uur gecompenseerd.
- KNMI: Tijdstippen in UTC

**Uitleg kolommen van KNMI**

Het KNMI levert veel gegevens aan, helaas zitten niet al deze gegevens in de weersvoorspelling. De vetgedrukte items, temperatuur, duur van de neerslag en bewolkingsgraad worden overgenomen. De rest van de kolommen worden hier niet verder gebruikt. 

- DD: Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil 990=veranderlijk. Meer info

- FH: Uurgemiddelde windsnelheid (in 0.1 m/s). Meer info

- FF: Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur

- FX: Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak

- **T**: Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming

- T10N: Minimumtemperatuur (in 0.1 graden Celsius) op 10 cm hoogte in de afgelopen 6 uur

- TD: Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming

- SQ: Duur van de zonneschijn (in 0.1 uren) per uurvak, berekend uit globale straling (-1 for <0.05 uur)

- Q: Globale straling (in J/cm2) per uurvak

- **DR**: Duur van de neerslag (in 0.1 uur) per uurvak

- RH: Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)

- P: Luchtdruk (in 0.1 hPa) herleid naar zeeniveau, tijdens de waarneming

- VV: Horizontaal zicht tijdens de waarneming (0=minder dan 100m, 1=100-200m, 2=200-300m,..., 49=4900-5000m, 50=5-6km, 56=6-7km, 57=7-8km, ..., 79=29-30km, 80=30-35km, 81=35-40km,..., 89=meer dan 70km)

- **N**: Bewolking (bedekkingsgraad van de bovenlucht in achtsten), tijdens de waarneming (9=bovenlucht onzichtbaar)

- U: Relatieve vochtigheid (in procenten) op 1.50 m hoogte tijdens de waarneming

- WW: Weercode (00-99), visueel(WW) of automatisch(WaWa) waargenomen, voor het actuele weer of het weer in het afgelopen uur. Meer info

- IX: Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen, 2,3=bemand en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 4=automatisch en opgenomen (gebruikmakend van code uit visuele waarnemingen), 5,6=automatisch en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 7=automatisch gebruikmakend van code uit automatische waarnemingen)

- M: Mist 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- R: Regen 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- S: Sneeuw 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- O: Onweer 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- Y: IJsvorming 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming



In [1]:
import sqlite3
import pysolar
import pandas as pd
import datetime

# Inlezen gegevens solaredge
def inlezen_solaredge():
    # Database inlezen
    conn = sqlite3.connect("solaredge.db")
    data = pd.read_sql('SELECT * FROM history', conn)
    
    # Tijd omzetten naar datetime
    # Compenseren voor UTC vs Europe/Amsterdam tijd
    data["Time"] = pd.to_datetime(data["tijdstip"], utc=True) + pd.Timedelta(hours=-1)
    data.set_index("Time", inplace=True)
    data.drop("tijdstip", axis=1, inplace=True)
    
    return data

# Inlezen gegevens weer
def inlezen_weer():
    # Database inlezen
    conn = sqlite3.connect("knmi.db")
    data = pd.read_sql('SELECT * FROM knmi', conn)
    
    # Date en hour omzetten naar datetime
    data["Time"] = pd.to_datetime(data["date"]) + pd.TimedeltaIndex(data["hour"], unit='h')
    data.set_index("Time", inplace=True)
    data.drop(["date", "hour", "id", "station_code"], inplace=True, axis=1)

    # De temperatuur staat in 0.1°C, deze wordt nu door 10 gedeeld om de
    # goede temperatuur te krijgen
    data["T"] = data["T"] / 10
    return data


# Zonnepanelen worden per kwartier gesampled. Met _resample_ het totaal per uur uitrekenen
zonnepanelen = inlezen_solaredge().resample("1H").sum()
    
# Data van het weer inlezen. Samenvoegen met data zonnepanelen
data = inlezen_weer()
data = data.merge(zonnepanelen, left_index=True, right_index=True)

# Nu ook de positie van de zon berekenen op basis van mijn positie
latitude = 51.2
longitude = 6

# pysolar heeft een datetime nodig. Met reset_index van de tijd een kolom maken,
# berekeningen uitvoeren en de tijd kolom weer als index terugzetten
data.reset_index(inplace=True)
data["alt"] = data.apply(lambda row: max(pysolar.solar.get_altitude(latitude, longitude, row["Time"].to_pydatetime()),0), axis=1)
data["azi"] = data.apply(lambda row: pysolar.solar.get_azimuth(latitude, longitude, row["Time"].to_pydatetime()), axis=1)
data.set_index("Time", inplace=True)

# Kolommen een begrijpelijke naam geven
data.rename(columns={"T": "temperatuur", 
             "DR": "duur_neerslag", 
             "N": "bewolking", 
             "alt": "solar_altitude", 
             "azi": "solar_azimuth"}, inplace=True)

# Dataset opslaan voor te gebruiken in machine learn model
conn = sqlite3.connect("dataset.db")
data[["temperatuur", "duur_neerslag", "bewolking", "solar_altitude", "solar_azimuth", "energy"]].to_sql('history', con=conn, if_exists='replace')
conn.commit()
conn.close()

In [2]:
data

,DD,FH,FF,FX,temperatuur,T10N,TD,SQ,Q,duur_neerslag,...,WW,IX,M,R,S,O,Y,energy,solar_altitude,solar_azimuth
Time,,,,,,,,,,,,,,,,,,,,,
2019-09-16 01:00:00+00:00,0,0,0,10,11.9,NaN,116,0,0,0,...,10.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.706222
2019-09-16 02:00:00+00:00,0,10,0,20,11.4,NaN,111,0,0,0,...,20.0,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,42.904684
2019-09-16 03:00:00+00:00,180,10,10,10,10.7,NaN,104,0,0,0,...,10.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.293104
2019-09-16 04:00:00+00:00,170,10,10,20,11.7,NaN,114,0,0,0,...,10.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.214002
2019-09-16 05:00:00+00:00,320,10,10,10,11.9,NaN,116,0,0,0,...,10.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82.227594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-30 22:00:00+00:00,210,30,40,60,4.5,NaN,7,0,1,0,...,NaN,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317.178305
2022-01-30 23:00:00+00:00,210,50,50,70,4.9,NaN,8,0,1,0,...,NaN,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,339.290508
2022-01-30 23:00:00+00:00,210,50,50,70,4.9,NaN,8,0,1,0,...,NaN,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,339.290508
